# Adding Prompt & Resource Features

As we have already playaround the MCP Server and Client, To interact with multiple tools with different servers to access the data or some functionality, we can use it within the MCP Client.
Here in this below example, We are implementing a MCP Server with a resources and prompts to access the any data[arvix data] and then using the MCP Client to interact with it.

In [ ]:
%%writefile mcp_project/research_server_with_resource.py
import arxiv
import json
import os
from typing import List
from mcp.server.fastmcp import FastMCP


PAPER_DIR = "papers"

# Initialize FastMCP server
mcp = FastMCP("research")

@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

@mcp.tool()
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."

# Above tools are already we see in the MCP server, so we can use them directly. and Below we need to add the resource to the MCP server

# RESOURCES
@mcp.resource("papers://folders")
def get_available_folders() -> str:
    """
    List all available topic folders in the papers directory.
    
    This resource provides a simple list of all available topic folders.
    """
    folders = []
    
    # Get all topic directories
    if os.path.exists(PAPER_DIR):
        for topic_dir in os.listdir(PAPER_DIR):
            topic_path = os.path.join(PAPER_DIR, topic_dir)
            if os.path.isdir(topic_path):
                papers_file = os.path.join(topic_path, "papers_info.json")
                if os.path.exists(papers_file):
                    folders.append(topic_dir)
    
    # Create a simple markdown list
    content = "# Available Topics\n\n"
    if folders:
        for folder in folders:
            content += f"- {folder}\n"
        content += f"\nUse @{folder} to access papers in that topic.\n"
    else:
        content += "No topics found.\n"
    
    return content

@mcp.resource("papers://{topic}")
def get_topic_papers(topic: str) -> str:
    """
    Get detailed information about papers on a specific topic.
    
    Args:
        topic: The research topic to retrieve papers for
    """
    topic_dir = topic.lower().replace(" ", "_")
    papers_file = os.path.join(PAPER_DIR, topic_dir, "papers_info.json")
    
    if not os.path.exists(papers_file):
        return f"# No papers found for topic: {topic}\n\nTry searching for papers on this topic first."
    
    try:
        with open(papers_file, 'r') as f:
            papers_data = json.load(f)
        
        # Create markdown content with paper details
        content = f"# Papers on {topic.replace('_', ' ').title()}\n\n"
        content += f"Total papers: {len(papers_data)}\n\n"
        
        for paper_id, paper_info in papers_data.items():
            content += f"## {paper_info['title']}\n"
            content += f"- **Paper ID**: {paper_id}\n"
            content += f"- **Authors**: {', '.join(paper_info['authors'])}\n"
            content += f"- **Published**: {paper_info['published']}\n"
            content += f"- **PDF URL**: [{paper_info['pdf_url']}]({paper_info['pdf_url']})\n\n"
            content += f"### Summary\n{paper_info['summary'][:500]}...\n\n"
            content += "---\n\n"
        
        return content
    except json.JSONDecodeError:
        return f"# Error reading papers data for {topic}\n\nThe papers data file is corrupted."

# PROMPTS [With Dynamic Informations we pass as Arguments]
@mcp.prompt()
def generate_search_prompt(topic: str, num_papers: int = 5) -> str:
    """Generate a prompt for Claude to find and discuss academic papers on a specific topic."""
    return f"""Search for {num_papers} academic papers about '{topic}' using the search_papers tool. Follow these instructions:
    1. First, search for papers using search_papers(topic='{topic}', max_results={num_papers})
    2. For each paper found, extract and organize the following information:
       - Paper title
       - Authors
       - Publication date
       - Brief summary of the key findings
       - Main contributions or innovations
       - Methodologies used
       - Relevance to the topic '{topic}'
    
    3. Provide a comprehensive summary that includes:
       - Overview of the current state of research in '{topic}'
       - Common themes and trends across the papers
       - Key research gaps or areas for future investigation
       - Most impactful or influential papers in this area
    
    4. Organize your findings in a clear, structured format with headings and bullet points for easy readability.
    
    Please present both detailed information about each paper and a high-level synthesis of the research landscape in {topic}."""

if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting mcp_project/research_server_with_resource.py


In [2]:
%%writefile mcp_project/openai_chatbot_with_resource_and_prompts.py
from dotenv import load_dotenv
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from typing import TypedDict
from contextlib import AsyncExitStack
import json
import asyncio

import nest_asyncio

nest_asyncio.apply()

load_dotenv()


class ToolDefinition(TypedDict):
    name: str
    description: str
    parameters: dict
    type: str = "function"


class MCP_ChatBot:
    def __init__(self):
        self.exit_stack = AsyncExitStack()
        self.openai = OpenAI()
        self.available_tools = []
        self.available_prompts= []
        self.sessions= {}

    async def connect_to_server(self, server_name: str, server_config: dict) -> None:
        """Connect to a single MCP server."""
        try:
            server_params = StdioServerParameters(**server_config)
            stdio_transport = await self.exit_stack.enter_async_context(
                stdio_client(server_params)
            )
            read, write = stdio_transport
            session = await self.exit_stack.enter_async_context(
                ClientSession(read, write)
            )
            await session.initialize()

            try:
                # List available tools for this session
                response = await session.list_tools()
                tools = response.tools
                print(f"\nConnected to {server_name} with tools:", [t.name for t in tools])
                for tool in tools:
                    self.sessions[tool.name] = session
                    self.available_tools.append(
                        {
                            "type": "function",
                            "name": tool.name,
                            "description": tool.description,
                            "parameters": tool.inputSchema,
                        }
                    )
                
                for tool in self.available_tools:
                    tool["parameters"]["additionalProperties"] = False

                prompts_response = await session.list_prompts()
                if prompts_response and prompts_response.prompts:
                    print(f"\nConnected to {server_name} with prompts:", [p.name for p in prompts_response.prompts])
                    for prompt in prompts_response.prompts:
                        self.sessions[prompt.name] = session
                        self.available_prompts.append({
                            "name": prompt.name,
                            "description": prompt.description,
                            "arguments": prompt.arguments
                        })
                    

                # List Available resources for this session
                resources_response = await session.list_resources()
                if resources_response and resources_response.resources:
                    print(f"\nConnected to {server_name} with resources:", [str(r.uri) for r in resources_response.resources])
                    for resource in resources_response.resources:
                        resource_uri = str(resource.uri)
                        self.sessions[resource_uri] = session

            except Exception as e:
                print(f"Error {e}")

        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")

    async def connect_to_servers(self):  # new
        """Connect to all configured MCP servers."""
        try:
            with open("server_config_with_prompts_and_resource.json", "r") as file:
                data = json.load(file)

            servers = data.get("mcpServers", {})

            for server_name, server_config in servers.items():
                await self.connect_to_server(server_name, server_config)
        except Exception as e:
            print(f"Error loading server configuration: {e}")
            raise

    async def process_query(self, query: str):
        messages = [{"role": "user", "content": query}]

        is_loop_nedded = True
        while is_loop_nedded:
            response = self.openai.responses.create(
                model="gpt-4o",
                input=messages,
                tools=self.available_tools,
            )
            for block in response.output:
                if block.type == "message":
                    is_loop_nedded = False
                    print(block.content[0].text)

                elif block.type == "function_call":
                    tool_call_id = block.call_id
                    tool_name = block.name
                    tool_args = json.loads(block.arguments)

                    print(
                        f"Processing tool call: {tool_name} with args: {tool_args} with call_id: {tool_call_id}"
                    )

                    # Call a tool
                    session = self.sessions.get(block.name)
                    result = await session.call_tool(tool_name, arguments=tool_args)
                    messages.append(block.model_dump())
                    messages.append(
                        {
                            "type": "function_call_output",
                            "call_id": tool_call_id,
                            "output": str(result),
                        }
                    )
                    is_loop_nedded = True


    async def get_resource(self, resource_uri):
        session = self.sessions.get(resource_uri)

        # Fallback for papers URIs - try any papers resource session
        if not session and resource_uri.startswith("papers://"):
            for uri, sess in self.sessions.items():
                if uri.startswith("papers://"):
                    session = sess
                    break

        if not session:
            print(f"Resource '{resource_uri}' not found.")
            return

        try:
            result = await session.read_resource(uri=resource_uri)
            if result and result.contents:
                print(f"\nResource: {resource_uri}")
                print("Content:")
                print(result.contents[0].text)
            else:
                print("No content available.")
        except Exception as e:
            print(f"Error: {e}")

    async def list_prompts(self):
        """List all available prompts."""
        if not self.available_prompts:
            print("No prompts available.")
            return

        print("\nAvailable prompts:")
        for prompt in self.available_prompts:
            print(f"- {prompt['name']}: {prompt['description']}")
            if prompt['arguments']:
                print("  Arguments:")
                for arg in prompt['arguments']:
                    arg_name = arg.name if hasattr(arg, 'name') else arg.get('name', '')
                    print(f"    - {arg_name}")

    async def execute_prompt(self, prompt_name, args):
        """Execute a prompt with the given arguments."""
        session = self.sessions.get(prompt_name)
        if not session:
            print(f"Prompt '{prompt_name}' not found.")
            return

        try:
            result = await session.get_prompt(prompt_name, arguments=args)
            if result and result.messages:
                prompt_content = result.messages[0].content

                # Extract text from content (handles different formats)
                if isinstance(prompt_content, str):
                    text = prompt_content
                elif hasattr(prompt_content, 'text'):
                    text = prompt_content.text
                else:
                    # Handle list of content items
                    text = " ".join(item.text if hasattr(item, 'text') else str(item) 
                                  for item in prompt_content)

                print(f"\nExecuting prompt '{prompt_name}'...")
                await self.process_query(text)
        except Exception as e:
            print(f"Error: {e}")


    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")
        print("Use @folders to see available topics")
        print("Use @<topic> to search papers in that topic")
        print("Use /prompts to list available prompts")
        print("Use /prompt <name> <arg1=value1> to execute a prompt")

        while True:
            try:
                query = input("\nQuery :[Type 'quit' to exit]: ").strip()

                if query.lower() == "quit":
                    break

                # Check for @resource syntax first
                if query.startswith('@'):
                    # Remove @ sign  
                    topic = query[1:]
                    if topic == "folders":
                        resource_uri = "papers://folders"
                    else:
                        resource_uri = f"papers://{topic}"
                    await self.get_resource(resource_uri)
                    continue

                # Check for /command syntax
                if query.startswith('/'):
                    parts = query.split()
                    command = parts[0].lower()

                    if command == '/prompts':
                        await self.list_prompts()
                    elif command == '/prompt':
                        if len(parts) < 2:
                            print("Usage: /prompt <name> <arg1=value1> <arg2=value2>")
                            continue

                        prompt_name = parts[1]
                        args = {}

                        # Parse arguments
                        for arg in parts[2:]:
                            if '=' in arg:
                                key, value = arg.split('=', 1)
                                args[key] = value

                        await self.execute_prompt(prompt_name, args)
                    else:
                        print(f"Unknown command: {command}")
                    continue

                await self.process_query(query)

            except Exception as e:
                print(f"\nError: {str(e)}")

    async def cleanup(self):  # new
        """Cleanly close all resources using AsyncExitStack."""
        await self.exit_stack.aclose()


async def main():
    chatbot = MCP_ChatBot()
    try:
        # the mcp clients and sessions are not initialized using "with"
        # like in the previous lesson
        # so the cleanup should be manually handled
        await chatbot.connect_to_servers()  # new!
        await chatbot.chat_loop()
    finally:
        await chatbot.cleanup()  # new!


if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_project/openai_chatbot_with_resource_and_prompts.py


In [3]:
%%writefile mcp_project/server_config_with_prompts_and_resource.json
{
    "mcpServers": {
        "filesystem": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "."
            ]
        },
        "research": {
            "command": "uv",
            "args": [
                "run",
                "research_server_with_resource.py"
            ]
        },
        "fetch": {
            "command": "uvx",
            "args": [
                "mcp-server-fetch"
            ]
        }
    }
}

Overwriting mcp_project/server_config_with_prompts_and_resource.json


### How to run and Explore the Prompts and Resources

Step 1: Run the UV Virtual Environment to run MCP Client and Server with below steps

```bash
cd mcp_project # Before did this in your terminal, You need to go to our Repo's Root Directory

source .venv/bin/activate # Activate the virtual environment
```

Step 2: Run the MCP Server with below command

```bash
uv run openai_chatbot_with_resource_and_prompts.py
```

Now, play around with the MCP Client using the following command and review the example screenshots below:

After running the MCP Client, you can able to see the tools, propmts and resources available to use. from your own MCP Server and Also other references.
<img src="assets/mcp_execution_using_resource_and_prompts_1.png" width="auto" alt="MCP Inspector - Search Papers Tools">

You can able to get the resources list using below command:

Query : `@folders`

<img src="assets/mcp_execution_using_resource_and_prompts_2.png" width="auto" alt="MCP Inspector - Search Papers Tools">

And you can Able to get the resources using the resource name with this format.

Query : `@{resource_name}` - Here we able to get the entire information with in the `resource_name` folder. Example: `@@ai_agents`
<img src="assets/mcp_execution_using_resource_and_prompts_3.png" width="auto" alt="MCP Inspector - Search Papers Tools">

Below example is to list the available prompts in the MCP Server, which is used to interact with the MCP Server and get the data from the resources.

Query : `/prompts`

<img src="assets/mcp_execution_using_resource_and_prompts_4.png" width="auto" alt="MCP Inspector - Search Papers Tools">

Below one is to use the prompt to interact with the tools and resources available in the MCP Server.

Query : `/prompt`
Response : `Usage: /prompt <name> <arg1=value1> <arg2=value2>`

Query Example for using the prompt:

```bash
/prompt generate_search_prompt topic=love limit=5
```

<img src="assets/mcp_execution_using_resource_and_prompts_5.png" width="auto" alt="MCP Inspector - Search Papers Tools">

### How to Run and Explore the Prompts and Resources

**Step 1:** Run the UV Virtual Environment to execute the MCP Client and Server with the steps below:

```bash
cd mcp_project # Before running this command, make sure you are in the root directory of the repository

source .venv/bin/activate # Activate the virtual environment
```

**Step 2:** Run the MCP Server with the following command:

```bash
uv run openai_chatbot_with_resource_and_prompts.py
```

Now, you can experiment with the MCP Client using the following command and review the example screenshots below:

After running the MCP Client, you will be able to see the tools, prompts, and resources available from your own MCP Server as well as other references. <img src="assets/mcp_execution_using_resource_and_prompts_1.png" width="auto" alt="MCP Inspector - Search Papers Tools">

You can retrieve the list of resources using the following command:

**Query:** `@folders`

<img src="assets/mcp_execution_using_resource_and_prompts_2.png" width="auto" alt="MCP Inspector - Search Papers Tools">

You can also retrieve resources by using the resource name in the following format:

**Query:** `@{resource_name}` — This will return all the information within the specified `resource_name` folder.

Example: `@ai_agents` <img src="assets/mcp_execution_using_resource_and_prompts_3.png" width="auto" alt="MCP Inspector - Search Papers Tools">

The example below lists all the available prompts in the MCP Server, which you can use to interact with the server and retrieve data from the resources.

**Query:** `/prompts`

<img src="assets/mcp_execution_using_resource_and_prompts_4.png" width="auto" alt="MCP Inspector - Search Papers Tools">

The following query demonstrates how to use a prompt to interact with the available tools and resources in the MCP Server.

**Query:** `/prompt`

**Response:**

```
Usage: /prompt <name> <arg1=value1> <arg2=value2>
```

Example query for using a prompt:

```bash
/prompt generate_search_prompt topic=love limit=5
```

<img src="assets/mcp_execution_using_resource_and_prompts_5.png" width="auto" alt="MCP Inspector - Search Papers Tools">

> NOTE: Here’s the wrap-up in a short, simple points format:

- We ran the _MCP Server and MCP Client_
- We explored _tools_ available in the MCP Server
- We listed and _fetched resources_
- We viewed all _prompts_
- We used _prompts with arguments_ to get results

Now you know how to use **tools, prompts, and resources** via the _MCP Client and Server_! Tada! 🎉
